In [37]:
import re
import numpy as np
import pandas as pd


# O Paradigma Funcional

Vamos recapitular quais paradigmas de programação utilizamos até aqui:

**Paradigma Imperativo**
- O programa é composto por um estado e instruções que modificam este estado:

```python
x = 0
for i in range(10):
    x = (x + i)*2
print x
```
- Forma mais simples de programação, é o paradigma das linguagens mais antigas (C, Fortran e COBOL por exemplo).

**Orientação à Objetos (OOP)**
- O estado e as instruções que operam sobre esse estado são parte de um **objeto**:

```python
class Casa:
    def __init__(self):
        self.porta = 'Aberta'
    
    def fechar_porta(self):
        self.porta = 'Fechada'

    def abrir_porta(self):
        self.porta = 'Aberta'
```
- OOP é um paradigma muito comum hoje em dia, presente em linguagens como C++, Java e Python.

**Functional Programming**
- Não existe estado, apenas funções: o programa é uma sequência de aplicação de funções ao input:
```python
def somar_2(x):
    return x + 2
```
```python
def mult_4(x):
    return x * 4
```
```python
saida = somar_2(mult_4(somar_2(entrada)))
```
- No paradigma funcional funções são variáveis.
- Paradigma que surgiu com a linguagem LISP, nos anos 70, e hoje em dia sobrevive nas áreas de dados através de linguagens como R, Julia, Python (em parte).

## Funções são variáveis

Mesmo quando consturímos uma função utilizando `def`, o resultado é uma variável com uma função dentro!

In [38]:
soma_1 = lambda x: x + 1


In [39]:
soma_1


<function __main__.<lambda>(x)>

In [41]:
soma_1(1)

2

In [42]:
def soma_1_c(x):
    return x + 1


In [44]:
soma_1_c(1)


2

In [45]:
soma_1(soma_1_c(10))


12

Podemos utilizar esse comportamento para criar funções a partir de outras funções:

In [46]:
somar_n = lambda x, n: x + n


In [47]:
soma_1 = lambda x: somar_n(x, 1)


In [48]:
soma_1(10)


11

## O Conceito de `map`

Já vimos que podemos mapear uma função à uma `Series` do Pandas através do método `.map()`. Agora veremos como fazer a mesma coisa para qualquer iterável.

In [49]:
lista_exemplo = [10, 12, 34, 23, 2, 6, 7]


In [50]:
def div_2(x):
    return x / 2


**Como podemos aplicar a função `div_2` acima à `lista_exemplo`?**

In [51]:
div_2(lista_exemplo)


TypeError: unsupported operand type(s) for /: 'list' and 'int'

Não conseguimos aplicar a função diretamente: afinal, `div_2` espera receber 1 argumento inteiro, e `lista_exemplos` é uma lista!

Podemos utilizar programação imperativa e construir um loop:

In [53]:
new_list = []

for item in lista_exemplo:
    new_list.append(div_2(item))
print(lista_exemplo)
print(new_list)


[10, 12, 34, 23, 2, 6, 7]
[5.0, 6.0, 17.0, 11.5, 1.0, 3.0, 3.5]


Outra forma, já da programação funcional, são os lists comprehensions:

In [54]:
[div_2(item) for item in lista_exemplo]


[5.0, 6.0, 17.0, 11.5, 1.0, 3.0, 3.5]

Uma terceira forma é através da **função** `map()`:

In [55]:
map(div_2, lista_exemplo)

In [56]:
for i in map(div_2, lista_exemplo):
    print(i)


5.0
6.0
17.0
11.5
1.0
3.0
3.5


In [58]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

O resultado de `map()` é preguiçoso (*lazy*): isso significa que ele não é calculado até ser chamado diretamente. No exemplo acima utilizamos um loop para chamar cada elemento do resultado. Podemos transformar o resultado em uma lista, que forçará o Python a avaliar todas as execuções da função `div_2()`:

In [59]:
list(map(div_2, lista_exemplo))


[5.0, 6.0, 17.0, 11.5, 1.0, 3.0, 3.5]

Um comportamento curioso dos iteradores preguiçosos é que eles se esvaziam conforme chamamos os seus elementos:

In [60]:
resultado_map = map(div_2, lista_exemplo)
for i in resultado_map:
    print(i)


5.0
6.0
17.0
11.5
1.0
3.0
3.5


In [61]:
resultado_map

Uma vez iterado, o objeto *esquece* os valores que:

In [64]:
import time

In [66]:
resultado_map = map(div_2, lista_exemplo)
for i in resultado_map:
    print(i)
    time.sleep(5)

5.0
6.0


KeyboardInterrupt: 

In [67]:
list(resultado_map)

[17.0, 11.5, 1.0, 3.0, 3.5]

### Lazy evaluation

A avaliação lazy é um conceito fundamental em Big Data: ela economiza memória e tempo de processamento realizando avaliações somente quando estas são necessárias.

In [68]:
lista_telefones = [
    19999571559,
    "(21) 2412-0107",
    "(34) 99762-1166",
    "91-4002-8282",
    "(19) 3542-1820",
    "(19) 3561-9525",
    "(34) 3333-5802",
]
pattern = r"[0-9]{2}"


In [71]:
re.findall(pattern, str(lista_telefones[0]))[0]

'19'

In [76]:
"".join(re.findall(pattern, str(lista_telefones[1])))

'2124120107'

In [72]:
lista_dds = list(map(lambda x: re.findall(pattern, str(x))[0], lista_telefones))
print(lista_dds)


['19', '21', '34', '91', '19', '19', '34']


In [73]:
for ddd in map(lambda x: "".join(re.findall(pattern, str(x)))[:2], lista_telefones):
    print(ddd)

19
21
34
91
19
19
34


In [74]:
list(map(lambda x: "".join(re.findall(pattern, str(x)))[:2], lista_telefones))

['19', '21', '34', '91', '19', '19', '34']

## Filtrando iteráveis com `filter()`

A segunda parte importante do paradigma funcional é a função `filter()`: ela nos permite filtrar os elementos de um iterável a partir de uma função que retorna valores booleanos. Assim como `map()`, `filter()` avalia (de forma preguiçosa) um iterável e retorna apenas os elementos onde a função aplicada retorna `True`.

Vamos continuar o nosso exemplo com uma lista de telefones e uma função para extrair o DDD:

In [77]:
lista_telefones = [
    19999571559,
    "(21) 2412-0107",
    "(34) 99762-1166",
    "91-4002-8282",
    "(19) 3542-1820",
    "(19) 3561-9525",
    "(34) 3333-5802",
]


def extrair_ddd(telefone):
    """
    Recebe um telefone e retorna seu DDD

    telefone (str or int): Telefone onde os dois primeiros digitios numéricos são o DDD
    """
    pattern = r"[0-9]{2}"
    return "".join(re.findall(pattern, str(telefone)))[:2]


In [80]:
lista_19 = filter(lambda x: extrair_ddd(x) == "19", lista_telefones)
for telefone in lista_19:
    print(telefone)

19999571559
(19) 3542-1820
(19) 3561-9525


In [81]:
lista_ddd_19 = list(filter(lambda x: extrair_ddd(x) == "19", lista_telefones))
print(lista_ddd_19)


[19999571559, '(19) 3542-1820', '(19) 3561-9525']


Assim como o `map()`, podemos replicar o comportamento do `filter()` com uma list comprehension. A diferença é que tanto `map()` quanto `filter()` são *lazy*!

In [83]:
[telefone for telefone in lista_telefones if extrair_ddd(telefone) == "19"]


[19999571559, '(19) 3542-1820', '(19) 3561-9525']

# Voltamos 9h43!

## Agregando iteráveis com `reduce()`

A função `reduce()` cria a idéia de um `accumulator`. Imagine uma função `sum(x, y)` que calcula a soma de x e y. Podemos utilizar a função `reduce()` (da biblioteca `functools`) para tratar x como o `accumulator` e percorrer o iterador *acumulando* a aplicação da função.

Vamos considerar a lista `[1,4,6,8]` e função soma:

```python
def sum_two_elements(a,b):
    return a+b
```

podemos utilizar a função `reduce()` para somar os elementos de toda a lista:

```python
reduce( sum_two_elements, [1,4,6,8] )
```

Vamos ver o que cada etapa do `reduce()` foi:

```python
a = 0 # accumulator
b = 1 # value
a + b = 1 # so the accumulator receives this cummulative sum

a = 1 # accumulator
b = 4 # value
a + b = 5
...
a = 5 # accumulator
b = 6 # value 
a + b = 11
...
a = 11 # accumulator
b = 8 # value
a + b = 19

return 19
```

In [84]:
from functools import reduce


### Exemplo 1: Números

In [89]:
def somar_ab(a, b):
    print(f"a={a}, b={b}")
    return a + b


In [90]:
lista_numeros = [1, 4, 6, 8]
reduce(somar_ab, lista_numeros)


a=1, b=4
a=5, b=6
a=11, b=8


19

In [91]:
def comp_ab(x, y):
    print(f"a={x}, b={y}")
    if x > y:
        return x
    else:
        return y


reduce(comp_ab, [2, 10, 25, 1, -10, 13, 40, 20])


a=2, b=10
a=10, b=25
a=25, b=1
a=25, b=-10
a=25, b=13
a=25, b=40
a=40, b=20


40

### Exemplo 2: Strings

In [92]:
lista_letras = ["P", "e", "d", "r", "o"]


In [93]:
reduce(lambda x, y: x + y, lista_letras)


'Pedro'

Um exemplo prático é a utilização do reduce para selecionar o maior string de uma lista:

In [94]:
lista_nomes = ["Amapá", "Roraima", "Pará", "Piauí", "Maranhão"]
reduce(lambda x, y: x if len(x) > len(y) else y, lista_nomes)


'Maranhão'

### Vendo as aplicações: `accumulate()`

Nos exemplos acima utilizamos a função `print()` para ver o resultado de cada etapa do `reduce()`. Podemos utilizar a função `accumulate()` para ver o resultado de cada etapa como uma lista:

In [95]:
from itertools import accumulate


In [96]:
accumulate(lista_numeros, lambda x, y: x + y)

In [97]:
lista_numeros = [1, 2, 3, 4, 5]
list(accumulate(lista_numeros, lambda x, y: x + y))


[1, 3, 6, 10, 15]

In [98]:
lista_nomes = ["Amapá", "Roraima", "Pará", "Piauí", "Maranhão"]
list(accumulate(lista_nomes, lambda x, y: x if len(x) > len(y) else y))


['Amapá', 'Roraima', 'Roraima', 'Roraima', 'Maranhão']

# Paradigma Funcional no Pandas

Até agora vimos como podemos utilizar as funções `map()`, `filter()` e `reduce()` para aplicar funções à, filtrar e reduzir iteráveis. Embora essas funções sejam extremamente úteis no contexto de dados (principalmente quando lidamos com dados extraídos de APIs ou *webscrapping*), em geral vamos entrar em contato com esses conceitos através do método `.apply()` de DataFrames e Series.

Nos exemplos abaixo, utilizaremos esse método, primeiro sobre `Series`, onde ele se comporta exatamente como o método `.map()`.

In [99]:
file_address = "http://www.statsci.org/data/general/sleep.txt"
tb_sleep = pd.read_csv(file_address, sep="\t")
tb_sleep.head()


,Species,BodyWt,BrainWt,NonDreaming,Dreaming,TotalSleep,LifeSpan,Gestation,Predation,Exposure,Danger
0,Africanelephant,6654.000,5712.0,NaN,NaN,3.3,38.6,645.0,3,5,3
1,Africangiantpouchedrat,1.000,6.6,6.3,2.0,8.3,4.5,42.0,3,1,3
2,ArcticFox,3.385,44.5,NaN,NaN,12.5,14.0,60.0,1,1,1
3,Arcticgroundsquirrel,0.920,5.7,NaN,NaN,16.5,NaN,25.0,5,2,3
4,Asianelephant,2547.000,4603.0,2.1,1.8,3.9,69.0,624.0,3,5,4


## `.apply()` em Colunas (`Series`)

O método `.apply()`, quando executado a partir de uma `Series` (uma coluna individual) funciona exatamente como o método `.map()`: ele retorna uma nova `Series` (coluna) com o resultado da aplicação de nossa função elemento a elemento na coluna a partir da qual foi invocado:

In [100]:
tb_sleep["brain_wt_kg"] = tb_sleep["BrainWt"] / 1000
tb_sleep["ratio_brain"] = tb_sleep["brain_wt_kg"] / tb_sleep["BodyWt"]
tb_sleep[["brain_wt_kg", "ratio_brain"]].describe()


,brain_wt_kg,ratio_brain
count,62.000000,62.000000
mean,0.283134,0.009624
std,0.930279,0.008915
min,0.000140,0.000858
25%,0.004250,0.003103
50%,0.017250,0.006611
75%,0.166000,0.013668
max,5.712000,0.039604


Vamos definir uma função para classificar a proporção entre o peso do cérebro e o peso total de cada animal:

In [101]:
def maior_1p(ratio):
    if ratio > 0.01:
        return "Pesado"
    else:
        return "Leve"


Como podemos aplicar essa função à coluna `ratio_brain`?

In [102]:
maior_1p(tb_sleep["ratio_brain"])


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

A aplicação direta não funciona, afinal a função espera um número e `tb_sleep["ratio_brain"]` é uma `Series`!

Vamos utilizar o `.apply()` de `Series` (equivalente ao `.map()`):

In [103]:
tb_sleep["ratio_brain"].map(maior_1p)

0       Leve
1       Leve
2     Pesado
3       Leve
4       Leve
       ...  
57      Leve
58    Pesado
59    Pesado
60      Leve
61      Leve
Name: ratio_brain, Length: 62, dtype: object

In [104]:
tb_sleep["heavy_brain"] = tb_sleep["ratio_brain"].apply(maior_1p)


In [105]:
tb_sleep["heavy_brain"].describe()


count       62
unique       2
top       Leve
freq        41
Name: heavy_brain, dtype: object

## `.apply()` em Tabelas (`DataFrame`)

Podemos invocar o método `.apply()` a partir de DataFrames também! Enquanto o método `.apply()` de coluna passa para a função o **valor de cada observação**, o método `.apply()` de tabelas pode passar linhas ou colunas (dependendo do `axis` que escolhermos).

Vamos começar analisando o comportamento desse método utilizando uma função comum do Python: `max`.

### Exemplo 1: Máximo entre colunas

In [107]:
tb_sleep[["Predation", "Exposure", "Danger"]]

,Predation,Exposure,Danger
0,3,5,3
1,3,1,3
2,1,1,1
3,5,2,3
4,3,5,4
...,...,...,...
57,3,1,3
58,3,2,2
59,4,3,4
60,2,1,1


In [106]:
tb_sleep[["Predation", "Exposure", "Danger"]].apply(max)


Predation    5
Exposure     5
Danger       5
dtype: int64

O comportamento padrão do `.apply()` de `DataFrames` é aplicar a função sobre as colunas: no caso acima ele retornou o máximo de cada coluna `["Predation", "Exposure", "Danger"]`.

E se quisermos calcular, para cada observação, qual o máximo entre essas três colunas?

In [108]:
tb_sleep[["Predation", "Exposure", "Danger"]].apply(max, axis=1)


0     5
1     3
2     1
3     5
4     5
     ..
57    3
58    3
59    4
60    2
61    3
Length: 62, dtype: int64

In [109]:
tb_sleep["max_risco"] = tb_sleep[["Predation", "Exposure", "Danger"]].apply(max, axis=1)
tb_sleep[["Predation", "Exposure", "Danger", "max_risco"]].head()


,Predation,Exposure,Danger,max_risco
0,3,5,3,5
1,3,1,3,3
2,1,1,1,1
3,5,2,3,5
4,3,5,4,5


In [112]:
tb_sleep[["Predation", "Exposure", "Danger"]].min(axis = 1)

0     3
1     1
2     1
3     2
4     3
     ..
57    1
58    2
59    3
60    1
61    1
Length: 62, dtype: int64

### Exemplo 2: `.apply()` e funções Lambda

No exemplo acima a função `max` recebeu o iterável `[3, 5, 3]` na primeira linha e calculou o máximo entre eles. No entanto muitas vezes queremos aplicar mais do que uma função simples. 

Por exemplo, podemos calcular a diferença entre o risco máximo e o risco mínimo. Para tanto, utilizaremos uma função lambda:

In [113]:
tb_sleep["var_risco"] = tb_sleep[["Predation", "Exposure", "Danger"]].apply(
    lambda x: max(x) - min(x), axis=1
)
tb_sleep[["Predation", "Exposure", "Danger", "max_risco", "var_risco"]].head()


,Predation,Exposure,Danger,max_risco,var_risco
0,3,5,3,5,2
1,3,1,3,3,2
2,1,1,1,1,0
3,5,2,3,5,3
4,3,5,4,5,2


Para visualizarmos o que está acontecendo, **precisamos entender o que o método `.apply()` esta passando como `x` para nossa função lambda**!

### Exemplo 3: `.apply()` com argumentos

Nos exemplos acima aplicamos uma função diretamente sobre uma linha (`max` e `min`) - já que estas funções transformam iteráveis em números.

Outra aplicação comum é passar colunas específicas como argumentos particulares de uma função.

In [115]:
def diff_prop(a, b):
    '''
    Calcula (x-y)/(x+y).
    Arguments:
        x Float
        y Float
    Returns:
        Float
    '''
    return (a-b)/(a+b)

In [116]:
tb_sleep.apply(
    lambda x: diff_prop(x['NonDreaming'], x['Dreaming']), axis=1
)


0          NaN
1     0.518072
2          NaN
3          NaN
4     0.076923
        ...   
57    0.814815
58    0.670886
59    0.883495
60    0.319588
61         NaN
Length: 62, dtype: float64

In [117]:
tb_sleep["PropNonDreaming"] = tb_sleep.apply(
    lambda x: diff_prop(x['NonDreaming'], x['Dreaming']), axis=1
)
tb_sleep[["NonDreaming", "Dreaming", "PropNonDreaming"]]

,NonDreaming,Dreaming,PropNonDreaming
0,NaN,NaN,NaN
1,6.3,2.0,0.518072
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,2.1,1.8,0.076923
...,...,...,...
57,4.9,0.5,0.814815
58,13.2,2.6,0.670886
59,9.7,0.6,0.883495
60,12.8,6.6,0.319588


No exemplo acima utilizamos o fato que `x` nada mais que uma linha de nosso DataFrame. Podemos utilizar o **desempacotamento de argumentos** para realizar a mesma operação:

In [118]:
tb_sleep[["NonDreaming", "Dreaming"]].apply(lambda x: diff_prop(*x), axis=1)


0          NaN
1     0.518072
2          NaN
3          NaN
4     0.076923
        ...   
57    0.814815
58    0.670886
59    0.883495
60    0.319588
61         NaN
Length: 62, dtype: float64

In [121]:
tb_sleep[["NonDreaming", "Dreaming"]].apply(lambda x: x['Dreaming']/np.sum(x), axis = 1)

0          NaN
1     0.240964
2          NaN
3          NaN
4     0.461538
        ...   
57    0.092593
58    0.164557
59    0.058252
60    0.340206
61         NaN
Length: 62, dtype: float64

In [124]:
tb_sleep[["Predation", "Exposure", "Danger"]].apply(lambda x: np.std(x)/np.mean(x), axis = 1)

0     0.257130
1     0.404061
2     0.000000
3     0.374166
4     0.204124
        ...   
57    0.404061
58    0.202031
59    0.128565
60    0.353553
61    0.565685
Length: 62, dtype: float64

In [125]:
tb_sleep[["Predation", "Exposure", "Danger"]]

,Predation,Exposure,Danger
0,3,5,3
1,3,1,3
2,1,1,1
3,5,2,3
4,3,5,4
...,...,...,...
57,3,1,3
58,3,2,2
59,4,3,4
60,2,1,1


No caso acima, `x` é um iterável com os valores `NonDreaming` e `Dreaming` - `[6.3, 2.0]` na segunda linha de nosso DataFrame.

Utilizamos `*x` como argumento da função diff_prop, desempacotando esse iterável, passando seu primeiro elemento como o primeiro argumento da função e seu segundo elemento como segundo argumento da função.

Note que nesse caso, a ordem do filtro de colunas no inicio da linha (`tb_sleep[["NonDreaming", "Dreaming"]].apply()`) é determinante no resultado!

In [126]:
tb_sleep['non_prop'] = tb_sleep[["NonDreaming", "Dreaming"]].apply(lambda x: diff_prop(*x), axis=1)
tb_sleep['dre_prop'] = tb_sleep[["Dreaming", "NonDreaming"]].apply(lambda x: diff_prop(*x), axis=1)
tb_sleep[["NonDreaming", "Dreaming", "non_prop", "dre_prop"]].head()


,NonDreaming,Dreaming,non_prop,dre_prop
0,NaN,NaN,NaN,NaN
1,6.3,2.0,0.518072,-0.518072
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,2.1,1.8,0.076923,-0.076923


## Exemplo 4: `.apply()` com strings

In [127]:
lista_telefones = [
    999571559,
    "2412-0107",
    "99762-1166",
    "4002-8282",
    "3542-1820",
    "9-3561-9525",
    "3333-5802"
]
lista_ddd = [
    19,
    "(11)",
    "(33)",
    "16",
    "29",
    12,
    "(34)",
]
tb_telefone = pd.DataFrame({"ddd" : lista_ddd, "telefone" : lista_telefones})
tb_telefone


,ddd,telefone
0,19,999571559
1,(11),2412-0107
2,(33),99762-1166
3,16,4002-8282
4,29,3542-1820
5,12,9-3561-9525
6,(34),3333-5802


In [128]:
def limpar_telefone(ddd, telefone):
    """
    Extrai o DDD de um telefone
    telefone (str or numeric): telefone onde os dois primeiros digitos numéricos são o DDD
    """
    pattern = r"[0-9]+"
    ddd_limpo = "".join(re.findall(pattern, str(ddd)))
    tel_limpo = "".join(re.findall(pattern, str(telefone)))
    return f"({ddd_limpo}){tel_limpo}"


In [133]:
limpar_telefone("19", '35-61-3870')


'(19)35613870'

In [134]:
tb_telefone["telefone_completo"] = tb_telefone[["ddd", "telefone"]].apply(lambda x: limpar_telefone(*x), axis = 1)
tb_telefone

,ddd,telefone,telefone_completo
0,19,999571559,(19)999571559
1,(11),2412-0107,(11)24120107
2,(33),99762-1166,(33)997621166
3,16,4002-8282,(16)40028282
4,29,3542-1820,(29)35421820
5,12,9-3561-9525,(12)935619525
6,(34),3333-5802,(34)33335802


## Exemplo 5: `.apply()` com argumentos fixos

Nos exemplos acimas, todos os argumentos das funções aplicadas originavam em colunas do DataFrame. Algumas vezes queremos *fixar* um dos parâmetros de uma função ao aplica-la sobre nosso DataFrame

In [135]:
def maior_custom(valor, patamar):
    if valor > patamar:
        return "Pesado"
    else:
        return "Leve"


In [136]:
tb_sleep["ratio_brain"].apply(maior_custom)


TypeError: maior_custom() missing 1 required positional argument: 'patamar'

In [137]:
tb_sleep["ratio_brain"].map(lambda x: maior_custom(x, 0.005))


0       Leve
1     Pesado
2     Pesado
3     Pesado
4       Leve
       ...  
57    Pesado
58    Pesado
59    Pesado
60      Leve
61      Leve
Name: ratio_brain, Length: 62, dtype: object

# Voltamos 10h40!